In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import scipy as sc

## Importing all the data

In [2]:
all_data=pd.read_csv("C:\\Users\\HP\\Documents\\GitHub\\Wind_Power\\Data\\Data_for_this_paper\\Table_Training_Complete.csv",encoding='utf-8')

In [3]:
def data(beg,end):
    l=[col[0]]
    l.extend(col[beg:end])
    return(all_data[l])

In [4]:
test=pd.read_csv("C:\\Users\\HP\\Desktop\\Classeur1.csv",encoding='utf-8',header=None)

In [5]:
col=all_data.columns.to_list()

In [6]:
time=data(1,146)

In [7]:
forecast=data(146,291)

In [8]:
adme=data(580,725)

In [9]:
forecast=forecast.iloc[:127]
adme=adme.iloc[:127]

### Constants

In [10]:
theta1=0.1
theta2=0.2
dt=0.0069
max_change=1.01
max_iter2=100
max_iter1=50
gamma=0.2
eps=0.018

In [11]:
def I(gamma,x,forecast):
    forecast_gamma=forecast.copy()
    m=forecast_gamma[x]>=gamma
    v=forecast_gamma[x]<=1-gamma
    forecast_gamma=forecast_gamma.where(m & v)
    #forecast_gamma=forecast_gamma.where(m | v,-1)
    forecast_gamma['Date']=forecast['Date']
    return(forecast_gamma)

In [12]:
def pe(eps,x,forecast):
    forecast_eps=forecast.copy()
    m=forecast[x]<eps
    v=forecast[x]>=1-eps
    forecast_eps=forecast_eps.where(~m,eps)
    forecast_eps=forecast_eps.where(~v,1-eps)
    forecast_eps['Date']=forecast['Date']
    return(forecast_eps)

In [13]:
def J(eps,x,forecast_eps):
    m=forecast_eps[x]==eps
    v=forecast_eps[x]==1-eps
    forecast_j=forecast_eps.where(m | v)
    #forecast_eps=forecast_eps.where(v)
    forecast_j['Date']=forecast_eps['Date']
    return(forecast_j)

In [14]:
def pe_dot(forecast,dt):
    col=forecast.columns.to_list()
    forecast_dot=pd.DataFrame(columns=col)
    forecast_dot['Date']=forecast['Date']
    del col[0]
    forecast_dot[col]=(forecast[col].shift(-1)-forecast[col])/dt
    forecast_dot=forecast_dot.drop(forecast_dot.index[len(forecast_dot)-1])
    return(forecast_dot)

# Moment Matching

## First moment

In [15]:
def first_moment(v,theta1,theta2,dt,N):
    if theta1==theta2:
        return(v*np.exp(-theta1*dt))
    else:
        m=np.zeros(N)
        m[0]=v
        ds=dt/N
        for i in range(1,N):
            m[i]=m[i-1]*(1-theta1+(theta2-theta1)*(i/N)*ds)
    return(m)

## Second moment

In [16]:
def second_moment(v,theta1,theta2,p1,p2,alpha,dt,N):
    m1=first_moment(v,theta1,theta2,dt,N)
    m2=np.zeros(N)
    m2[0]=v**2
    ds=dt/N
    if theta1!=theta2:
        for i in range(1,N):
            m1=m1*np.exp((i-1)/N)
            p=p1+(p2-p1)*((i-1)/N)
            m2[i]=m2[i-1]+2*theta1*ds*(-(1+alpha)*m2[i-1])+alpha*m1*(1-2*p)+alpha*p*(2-p)
    else:
        for i in range(1,N):
            theta=theta1+(theta2-theta1)*((i-1)/N)
            m1=m1*np.exp((i-1)/N)
            p=p1+(p2-p1)*((i-1)/N)
            m2[i]=m2[i-1]+2*theta1*ds*(-(1+alpha)*m2[i-1])+alpha*m1*(1-2*p)+alpha*p*(2-p)

## Density approximation

###  Parameters

In [17]:
def moment_matching(m1,m2):
    mu=m1
    sig2=m2-m1**2
    xi1=-((mu+1)*(mu**2+sig2-1))/(2*sig2)
    xi2=((mu-1)*(mu**2+sig2-1))/(2*sig2)
    return(xi1,xi2)

### Log density

In [18]:
def log_density(v,xi1,xi2):
    a=-1
    b=1
    return(-sc.betaln(xi1,xi2)+(xi1-1)*np.log((v-a)/(b-a))+(xi2-1)*np.log((b-v)/(b-a)))